### Integrated-gradient on IMDB dataset (PyTorch)

This is an example of the integrated-gradient method on text classification with a PyTorch model. If using this explainer, please cite the original work: https://github.com/ankurtaly/Integrated-Gradients.

In [3]:
import ast
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import sklearn
from sklearn.datasets import fetch_20newsgroups

from omnixai.data.text import Text
from omnixai.preprocessing.text import Word2Id
from omnixai.explainers.tabular.agnostic.L2X.utils import Trainer, InputData, DataLoader
from omnixai.explainers.nlp.specific.ig import IntegratedGradientText

We apply a simple CNN model for this text classification task. Note that the method `forward` has two inputs `inputs` (token ids) and `masks` (the sentence masks). For `IntegratedGradientText`, the first input of the model must be the token ids.

In [4]:
class TextModel(nn.Module):

    def __init__(self, num_embeddings, num_classes, **kwargs):
        super().__init__()
        self.num_embeddings = num_embeddings
        self.embedding_size = kwargs.get("embedding_size", 50)
        self.embedding = nn.Embedding(self.num_embeddings, self.embedding_size)
        self.embedding.weight.data.normal_(mean=0.0, std=0.01)
        
        hidden_size = kwargs.get("hidden_size", 100)
        kernel_sizes = kwargs.get("kernel_sizes", [3, 4, 5])
        if type(kernel_sizes) == int:
            kernel_sizes = [kernel_sizes]

        self.activation = nn.ReLU()
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(self.embedding_size, hidden_size, k, padding=k // 2) for k in kernel_sizes])
        self.dropout = nn.Dropout(0.2)
        self.output_layer = nn.Linear(len(kernel_sizes) * hidden_size, num_classes)

    def forward(self, inputs, masks):
        embeddings = self.embedding(inputs)
        x = embeddings * masks.unsqueeze(dim=-1)
        x = x.permute(0, 2, 1)
        x = [self.activation(layer(x).max(2)[0]) for layer in self.conv_layers]
        outputs = self.output_layer(self.dropout(torch.cat(x, dim=1)))
        if outputs.shape[1] == 1:
            outputs = outputs.squeeze(dim=1)
        return outputs

We use a `Text` object to represent a batch of texts/sentences. The package `omnixai.preprocessing.text` provides some transforms related to text data such as `Tfidf` and `Word2Id`.

In [5]:
# Load the training and test datasets
train_data = pd.read_csv('../data/imdb.csv')
n = int(0.8 * len(train_data))
x_train = Text(train_data["review"].values[:n])
y_train = train_data["sentiment"].values[:n].astype(int)
x_test = Text(train_data["review"].values[n:])
y_test = train_data["sentiment"].values[n:].astype(int)
class_names = ["negative", "positive"]
# The transform for converting words/tokens to IDs
transform = Word2Id().fit(x_train)

The preprocessing function converts a batch of texts into token IDs and the masks. The outputs of the preprocessing function must fit the inputs of the model.

In [6]:
max_length = 256
device = "cuda" if torch.cuda.is_available() else "cpu"

def preprocess(X: Text):
    samples = transform.transform(X)
    max_len = 0
    for i in range(len(samples)):
        max_len = max(max_len, len(samples[i]))
    max_len = min(max_len, max_length)
    inputs = np.zeros((len(samples), max_len), dtype=int)
    masks = np.zeros((len(samples), max_len), dtype=np.float32)
    for i in range(len(samples)):
        x = samples[i][:max_len]
        inputs[i, :len(x)] = x
        masks[i, :len(x)] = 1
    return inputs, masks

We now train the CNN model and evaluate its performance.

In [47]:
model = TextModel(
    num_embeddings=transform.vocab_size,
    num_classes=len(class_names)
).to(device)
try:
    path = './model.pth'
    model.load_state_dict(torch.load(path, weights_only=True))
    print("loaded")
except Exception as error:
    print(error)

Trainer(
    optimizer_class=torch.optim.AdamW,
    learning_rate=1e-3,
    batch_size=128,
    num_epochs=1,
).train(
    model=model,
    loss_func=nn.CrossEntropyLoss(),
    train_x=transform.transform(x_train),
    train_y=y_train,
    padding=True,
    max_length=max_length,
    verbose=True
)

path = './model.pth'
torch.save(model.state_dict(), path)

loaded
 |████████████████████████████████████████| 100.0% Complete, Loss 0.0080


In [34]:
model.eval()
data = transform.transform(x_test)
data_loader = DataLoader(
    dataset=InputData(data, [0] * len(data), max_length),
    batch_size=32,
    collate_fn=InputData.collate_func,
    shuffle=False
)
outputs = []
for inputs in data_loader:
    value, mask, target = inputs
    y = model(value.to(device), mask.to(device))
    outputs.append(y.detach().cpu().numpy())
outputs = np.concatenate(outputs, axis=0)
predictions = np.argmax(outputs, axis=1)
print('Test accuracy: {}'.format(
    sklearn.metrics.f1_score(y_test, predictions, average='binary')))

Test accuracy: 0.8854501216545012


To initialize `IntegratedGradientText`, we need to set the following parameters:

  - `model`: The model to explain, whose type is `tf.keras.Model` or `torch.nn.Module`.
  - `embedding_layer`: The embedding layer in the model, which can be `tf.keras.layers.Layer` or `torch.nn.Module`.
  - `preprocess_function`: The pre-processing function that converts the raw input data into the inputs of `model`. The first output of `preprocess_function` should be the token ids.
  - `mode`: The task type, e.g., `classification` or `regression`.
  - `id2token`: The mapping from token ids to tokens.

In [46]:
allwords = ""
explainer = IntegratedGradientText(
    model=model,
    embedding_layer=model.embedding,
    preprocess_function=preprocess,
    id2token=transform.id_to_word
)
x = Text([
"This movie was the cinematic equivalent of watching paint dry.",
"I laughed, I cried, I fell asleep - what an emotional rollercoaster!",
"The plot twists in this film were more predictable than a daytime soap opera.",
"Despite its flaws, there was something oddly charming about this movie's quirky protagonist.",
"Imagine taking the worst elements of every genre and combining them into one incoherent mess. That's this film in a nutshell.",
"I'm honestly not sure if I loved or hated this movie - it was that perplexingly strange.",
"The special effects were so breathtakingly realistic, I forgot I was watching a work of fiction.",
"I'd rather have root canal surgery than sit through this movie again.",
"A solid 'meh' from me. Completely forgettable, but not actively unenjoyable either.",
"This movie was the cinematic equivalent of a warm cup of milk - harmless, but not particularly exciting.",
])
rawExplanations = explainer.explain(x)
explanations = ast.literal_eval(str(rawExplanations))
result = []
for i in range(len(explanations)):
    tokens = explanations[i]["tokens"]
    scores = explanations[i]["scores"]
    append = []
    for j in range(len(tokens)):
        word = tokens[j]
        score = scores[j]
        if (int(explanations[i]["target_label"] == 0)):
            score *= float(-1)
        append.append([word, score])
    result.append(append)
totals = []
for i in range(len(result)):
    wordScores = result[i]
    total = 0
    for j in range(len(wordScores)):
        total += wordScores[j][1]
    totals.append(total)
result.append(totals)
print(result)
# print(sorted(result, key=lambda x: x[1]))
rawExplanations.ipython_plot(class_names=class_names)

[[['this', 0.24439932455804175], ['movie', 0.05638953305392258], ['was', -0.11364241896166592], ['the', 0.0734531548248518], ['cinematic', -0.05027196220877296], ['equivalent', -0.006102904416397761], ['of', 0.34827290113764003], ['watching', -0.2484394243985918], ['paint', -0.9119018231743818], ['dry', -1.1809996579923794]], [['i', -0.06134133863345417], ['laughed', 1.3757163551648801], ['i', 0.3416740493392372], ['cried', 4.314692952638511], ['i', 0.23247546696249063], ['fell', -2.8439235097262845], ['asleep', -2.367692449704707], ['what', 0.47426367554437415], ['an', 0.17344284351690134], ['emotional', 0.11976299364132414], ['rollercoaster', 0.00015693529280373653]], [['the', -0.0716057867043087], ['plot', -0.11605594109757189], ['twists', 0.26913418374024295], ['in', -0.12094787241463391], ['this', -0.04835812672748237], ['film', -0.0021218357937717175], ['were', 0.19803726134979066], ['more', 0.0573806224061085], ['predictable', -3.9000199498748174], ['than', -0.508941054289978], 